## RESEARCH QUESTION 2

To what extent past class failures might have an impact on alcohol consumption in students?

In [77]:
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline

from sklearn import set_config
set_config(transform_output = "pandas")

## Function to Save Figures

In [78]:
from pathlib import Path
IMAGES_PATH = Path() / "images"
IMAGES_PATH.mkdir(parents=True, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = IMAGES_PATH / f"{fig_id}.{fig_extension}"
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

In [79]:
# Import train and test sets
X_train = pd.read_csv('smoteTrainSet.csv')
X_test = pd.read_csv('ScaledTestSet.csv')


In [ ]:
X_train.columns


## baseline classifier 0 R

In [ ]:
from sklearn.dummy import DummyClassifier
dummy = DummyClassifier(strategy = "most_frequent")
X = X_train.drop(columns = 'alc')
y = X_train['alc']

In [ ]:
xt = X_test.drop('avg_grade',axis=1)

In [ ]:
dummy.fit(X,y)

### 0r classifier accuray

In [ ]:
from sklearn.metrics import accuracy_score
print('Train Accuracy', accuracy_score(X_train['alc'], dummy.predict(X)))
print('Test Accuracy', accuracy_score(X_test['alc'], 
                                      dummy.predict(xt)))

## 1 R classifier


In [ ]:
def one_r_classifier(X, y, X_test):
    best_feature = None
    best_accuracy = 0
    
    # Iterate over each feature
    for feature in X_train.columns:
        # Count the occurrences of each label for the current feature
        label_counts = X.groupby([feature, y]).size().unstack(fill_value=0)
        
        # Select the label with the maximum count for the current feature
        selected_label = label_counts.idxmax(axis=1)
        
        # Predict the label for the test set
        predictions = X_test[feature].map(selected_label.get).fillna(selected_label.value_counts().idxmax())
        
        # Calculate accuracy
        accuracy = accuracy_score(y, predictions)
        
        # Update the best feature if the accuracy is higher
        if accuracy > best_accuracy:
            best_feature = feature
            best_accuracy = accuracy
    print(best_feature)
    return best_feature



In [ ]:
best_feature = one_r_classifier(X, y, X_test)
print("Best feature:", best_feature)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

from sklearn_contrib_py_1R import OneRClassifier

# Create the 1R classifier and fit it to the training data
classifier = OneRClassifier()
classifier.fit(X, y)

# Make predictions on the testing data
y_pred = classifier.predict(X_test)

# Evaluate the classifier
accuracy = accuracy_score(X_test['alc'], y_pred)
print("Accuracy:", accuracy)

In [80]:
# Separate target from dataset
y_train = X_train['alc']
X_train = X_train[['schoolsup_no', 'schoolsup_yes', 'paid_no', 'paid_yes', 'higher_no', 'higher_yes', 'studytime', 'failures', 'absences', 'avg_grade']] 
y_test = X_test['alc']
X_test = X_test[['schoolsup_no', 'schoolsup_yes', 'paid_no', 'paid_yes', 'higher_no', 'higher_yes', 'studytime', 'failures', 'absences', 'avg_grade']]



In [81]:
y_train

0      1.0
1      1.0
2      0.0
3      1.0
4      1.0
      ... 
887    1.0
888    1.0
889    1.0
890    1.0
891    1.0
Name: alc, Length: 892, dtype: float64

## KNN Classifier

In [82]:
from sklearn.neighbors import KNeighborsClassifier
knn =KNeighborsClassifier(n_neighbors = 25)
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=25)

In [83]:
# Predicting Alcohol Consumption
y_pred = knn.predict(X_test)

In [84]:
pd.DataFrame({'y_test': y_test, 'y_pred': y_pred})

,y_test,y_pred
0,1.0,1.0
1,1.0,1.0
2,1.0,0.0
3,1.0,1.0
4,0.0,1.0
...,...,...
204,0.0,1.0
205,0.0,1.0
206,0.0,0.0
207,0.0,0.0


In [87]:
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
print('Test Accuracy ', accuracy_score(y_test, y_pred ))

Test Accuracy  0.5598086124401914


In [88]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.66      0.56      0.61       127
         1.0       0.45      0.56      0.50        82

    accuracy                           0.56       209
   macro avg       0.56      0.56      0.55       209
weighted avg       0.58      0.56      0.56       209



In [89]:
pd.DataFrame(confusion_matrix(y_test, y_pred))

,0,1
0,71,56
1,36,46


In [90]:
print('Train Accuracy ', accuracy_score(y_train, knn.predict(X_train) ))

Train Accuracy  0.6311659192825112


## DESCISION TREE

In [92]:
# Import train and test sets
X_train = pd.read_csv('smoteTrainSet.csv')
X_test = pd.read_csv('ScaledTestSet.csv')

In [93]:
x= X_train[['schoolsup_no', 'schoolsup_yes', 'paid_no', 'paid_yes', 'higher_no', 'higher_yes', 'studytime', 'failures', 'absences', 'avg_grade']]
y=X_train['alc']
y_test=X_test['alc']
X_test= X_test[['schoolsup_no', 'schoolsup_yes', 'paid_no', 'paid_yes', 'higher_no', 'higher_yes', 'studytime', 'failures', 'absences', 'avg_grade']]




In [94]:
from sklearn.metrics import accuracy_score
dt = DecisionTreeClassifier()
dt.fit(x, y)
print('Accuracy ', accuracy_score(y_test, dt.predict(X_test)).round(3))

Accuracy  0.579


In [96]:
from sklearn.tree import export_graphviz

export_graphviz(
        dt,
        out_file=str(IMAGES_PATH / "dec-tree-full.dot"), 
        feature_names=X_train.columns,
        class_names=['0','1'],
        rounded=True,
        filled=True
    )

ValueError: Length of feature_names, 43 does not match number of features, 10